# Build pipeline with sweep node

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create sweep node with `sweep()`
- Create `Pipeline` with sweep node

**Motivations** - This notebook explains how to create a sweep node by using `sweep()` and use it in a pipeline. A sweep node can be used to enable hyperparameter tuning on a specified compute (either local or on the cloud) for a specific command component. You can define a `search_space` and an `objective` to search for the target output.  

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [6]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml.sweep import (
    Choice,
    Uniform,
)

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [7]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb).

In [8]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "ej-cluster2"
print(ml_client.compute.get(cluster_name))

Found the config file in: /config.json


enable_node_public_ip: true
id: /subscriptions/876b91eb-54d6-4433-af3b-5c9914d5ccea/resourceGroups/ej_vision_playground/providers/Microsoft.MachineLearningServices/workspaces/ej-workshop-workspace/computes/ej-cluster2
idle_time_before_scale_down: 120
location: westeurope
max_instances: 6
min_instances: 1
name: ej-cluster2
provisioning_state: Succeeded
size: Standard_NC6s_v3
ssh_public_access_enabled: false
tier: low_priority
type: amlcompute



# 2. Pipeline job with hyperparameter sweep

## 2.1 Build pipeline
In pipeline definition, we define `search_space` for hyperparameter sweep in inputs of `train_model` and call `train_model.sweep()` to create a sweep node based on `train_model` with specific run settings. Run settings includes:
- objective_primary_metric 
- objective_goal
- sampling_algorithm
- limits
- early_termination_policy
- compute

Please check section **3. Run a sweep on this command** in [Run hyperparameter sweep on a Command or CommandComponent](../../single-step/lightgbm/iris/lightgbm-iris-sweep.ipynb) for detailed explanation for the above concepts and settings.

Noted that the **primary metric** of sweep objective must be **LOGGED** in the definition of `train_model`.

In [9]:
train_component_func = load_component(source="./train.yml")
score_component_func = load_component(source="./predict.yml")

# define a pipeline
@pipeline()
def pipeline_with_hyperparameter_sweep():
    """Tune hyperparameters using sample components."""
    train_model = train_component_func(
        data=Input(
            type="uri_file",
            path="wasbs://datasets@azuremlexamples.blob.core.windows.net/iris.csv",
        ),
        c_value=Uniform(min_value=0.5, max_value=0.9),
        kernel=Choice(["rbf", "linear", "poly"]),
        coef0=Uniform(min_value=0.1, max_value=1),
        degree=3,
        gamma="scale",
        shrinking=False,
        probability=False,
        tol=0.001,
        cache_size=1024,
        verbose=False,
        max_iter=-1,
        decision_function_shape="ovr",
        break_ties=False,
        random_state=42,
    )
    sweep_step = train_model.sweep(
        primary_metric="training_f1_score",
        goal="minimize",
        sampling_algorithm="random",
        compute="ej-cluster2",
    )
    sweep_step.set_limits(max_total_trials=20, max_concurrent_trials=10, timeout=7200)

    score_data = score_component_func(
        model=sweep_step.outputs.model_output, test_data=sweep_step.outputs.test_data
    )


pipeline_job = pipeline_with_hyperparameter_sweep()

# set pipeline level compute
pipeline_job.settings.default_compute = "ej-cluster2"

## 2.2 Submit pipeline job

In [10]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="hyper-param-sweep"
)
pipeline_job

Uploading train-src (0.0 MBs): 100%|██████████| 3610/3610 [00:00<00:00, 59380.51it/s]


Uploading predict-src (0.0 MBs): 100%|██████████| 969/969 [00:00<00:00, 14992.75it/s]




Experiment,Name,Type,Status,Details Page
hyper-param-sweep,serene_grape_z3tsbn13lm,pipeline,Preparing,Link to Azure Machine Learning studio


In [11]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

RunId: serene_grape_z3tsbn13lm
Web View: https://ml.azure.com/runs/serene_grape_z3tsbn13lm?wsid=/subscriptions/876b91eb-54d6-4433-af3b-5c9914d5ccea/resourcegroups/ej_vision_playground/workspaces/ej-workshop-workspace

Streaming logs/azureml/executionlogs.txt

[2024-03-20 09:16:51Z] Submitting 1 runs, first five are: 90a1dd90:7810fa46-a23d-4c2c-8711-f5aea2cef29b
[2024-03-20 09:29:09Z] Completing processing run id 7810fa46-a23d-4c2c-8711-f5aea2cef29b.
[2024-03-20 09:29:10Z] Submitting 1 runs, first five are: 4cf2b20a:06fd4f0a-0b27-4d4e-ab53-6883e160f3b8
[2024-03-20 09:32:04Z] Completing processing run id 06fd4f0a-0b27-4d4e-ab53-6883e160f3b8.

Execution Summary
RunId: serene_grape_z3tsbn13lm
Web View: https://ml.azure.com/runs/serene_grape_z3tsbn13lm?wsid=/subscriptions/876b91eb-54d6-4433-af3b-5c9914d5ccea/resourcegroups/ej_vision_playground/workspaces/ej-workshop-workspace



# Next Steps
You can see further examples of running a pipeline job [here](../)